Part 1: Read cell body position from .apo files, downsample and save them as txt file for applying inverse transformation

In [32]:
import Neuron_analysis as na
from Neuron_analysis import *

import os
import pandas as pd
import re

import tkinter as tk
import tkinter.filedialog as fdialog
from tkinter import simpledialog

In [68]:
indir = fdialog.askdirectory(title='Select the directories with all the cell bodies.apo')
outdir = fdialog.askdirectory(title='Select the output directory to store your file')

files = os.listdir(indir)

In [69]:
def prepare_cellbody(indir,anno_file,outdir):
    anno=open(os.path.join(indir,anno_file),'r')
    anno_data=anno.readlines()
 
    headings=anno_data[0].rstrip('\n').split(',')
    annotations=[lines.rstrip('\n').split(',') for lines in anno_data[1:]]
    annotation_df=pd.DataFrame(annotations, columns=headings)
    
    cell_body= annotation_df[annotation_df.color_g=='255']
    #pickout only green points!
    
    # find the coordinates and downsample it! these values are based on xy= 0.6 z= 5, and downsampling to 25 xyz
    ds_x= pd.to_numeric(cell_body['x'])//41
    ds_y= pd.to_numeric(cell_body['y'])//41
    ds_z= pd.to_numeric(cell_body['z'])//5

    x=ds_x.astype(int).astype(str)
    y=ds_y.astype(int).astype(str)
    z=ds_z.astype(int).astype(str)
    
    m=re.search('\D{2}[0-9]{3}', anno_file)
    mouse_name=m[0]

    out_name=os.path.join(outdir,mouse_name+'cellbody.txt')

    f=open(out_name,'w+')
    f.write('point'+'\n')
    f.write('1'+'\n') # by default there is only 1 cell body, hence only 1 point here
    f.write(x.values[0]+ ' ' + y.values[0]+ ' '+ z.values[0])
    f.close()
    return
    

In [70]:
for items in files:
    prepare_cellbody(indir, items, outdir)

part2: (after transformation by transformix) extract the transformed points, refill y section (because we cropped the template for registration). 
Then write as a single file containing brain ID and cell body location

In [24]:
brain_id= ['AL175', 'AL223','AL230', 'AL236', 'AL244', 'AL261']
#missing 'AL225' and 'AL242' for now
cb_file=f'D:\{brain_id[1]}\points\cellbody\outputpoints.txt'

In [60]:
all_points=[]

for items in brain_id:
    this_file=cb_file=f'D:\{items}\points\cellbody\outputpoints.txt'
    
    template_file = [x for x in os.listdir(f'D:\{items}\\') if 'template' in x]
    to_add=na.find_crop(template_file[0])
  
    with open(cb_file,'r') as output:
        outputpoint= output.readlines()

    for lines in outputpoint:
        m=re.search("(?:OutputIndexFixed = \[ )([0-9]+ [0-9]+ [0-9]+)", lines).groups(0)
        this_line= str(m[0]).split(' ')
        mypoints= [int(stuff) for stuff in this_line]
        mypoints[1]= mypoints[1]+to_add
        mypoints.insert(0, items)
        all_points.append(mypoints)
        
# the coordiates are arranged in x,y and z

In [59]:
all_points

[['AL175', 92, 218, 52],
 ['AL223', 99, 214, 47],
 ['AL230', 88, 210, 56],
 ['AL236', 104, 214, 40],
 ['AL244', 86, 237, 47],
 ['AL261', 123, 196, 42]]

In [61]:
all_points

[['AL175', 92, 283, 52],
 ['AL223', 99, 279, 47],
 ['AL230', 88, 275, 56],
 ['AL236', 104, 279, 40],
 ['AL244', 86, 302, 47],
 ['AL261', 123, 261, 42]]

In [50]:
to_add= na.find_crop(template_file[0])

65

In [51]:
mypoints

['AL261', 123, 196, 42]